## Indoor Location & Navigation

В этом соревновании ваша задача - предсказать положение смартфонов в помещении на основе данных датчиков в реальном времени, предоставленных компанией XYZ10, занимающейся технологиями внутреннего позиционирования, в партнерстве с Microsoft Research.

Metric
Submissions are evaluated on the mean position error as defined as:

$$\text{mean position error} = \frac{1}{N} \sum_{i=1}^{N}  
                                                \left( \sqrt{( \hat{x}_i - x_i )^{2} + ( \hat{y}_i - y_i )^{2}} 
                                                + p \cdot | \hat{f}_{i} - f_i | \right)$$
                                                
where:

- N is the number of rows in the test set
- $\hat{x}$,$\hat{y}$ are the predicted locations for a given test row
- x, y are the ground truth locations for a given test row
- p is the floor penalty, set at 15
- $\hat{f}$, f are the predicted and ground truth integer floor level for a given test row

IMPORTANT: The integer floor used in the submission must be mapped from the char/int floors used in the dataset. The mapping is as follows:

- F1, 1F  0
- F2, 2F  1
- etc.
- B1, 1B  -1
- B2, 2B  -2


Набор данных для этого соревнования состоит из плотных внутренних сигнатур:
- Wi-Fi,
- геомагнитного поля,
- iBeacons и т. Д.,

А также наземных данных (путевых точек) (местоположений), собранных из сотен зданий в китайских городах. Данные, содержащиеся в файлах трассировки пути (* .txt), соответствуют внутреннему пути между точками p_1 и p_2, пройденному геодезистом.

## Look to data

- TYPE_ACCELEROMETER
- TYPE_MAGNETIC_FIELD
- TYPE_GYROSCOPE
- TYPE_ROTATION_VECTOR
- TYPE_MAGNETIC_FIELD_UNCALIBRATED
- TYPE_GYROSCOPE_UNCALIBRATED
- TYPE_ACCELEROMETER_UNCALIBRATED
- TYPE_WIFI
- TYPE_BEACON
- TYPE_WAYPOINT: ground truth location labeled by the surveyor

```
data
  |___ /input/train/
          file1
            |___ floor(B1,F1....)
            |        B1       
            |        |___ trace_file
            |        |    5dda14a2c5b77e0006b17533.txt
            |        |        |     |___ types
            |        |        |         |
            |        |        |         TYPE_ACCELEROMETER
            |        |    meta_data          ...
            |        |        |___ 
            |        |             floor_image.png 
            |        |             floor_info.json  
            |        |             geojson_map.json
            |        F1                     
          file 2    ...
           ...
```


In [1]:
import glob, os, gc
import numpy as np
import pandas as pd
from IPython.core.display import HTML


path = '../input/train/'

In [17]:
def simple_data(typ: str, name: str, floor: str, file: str, columns: list):
    box = []
    with open(os.path.join(path, name, floor, file)) as f:
        txt = f.readlines()
        count = 0
        for line in txt:
            tmp = line.strip().split()
            if tmp[1] == typ:                
                box.append(tmp)
                count += 1
                if count == 5:
                    break
    display(HTML(pd.DataFrame(box, columns = columns).to_html()))             
    

In [18]:
name    = '5a0546857ecc773753327266'
floor   = 'B1'
file    = '5e15730aa280850006f3d005.txt'
columns = ['time', 'data tipe','X axis', 'Y axis', 'Z axis','accuracy']
typ     = 'TYPE_ACCELEROMETER'
simple_data(typ, name, floor, file, columns)

,time,data tipe,X axis,Y axis,Z axis,accuracy
0,1578462618653,TYPE_ACCELEROMETER,0.0236969,4.450943,9.055649,2
1,1578462618673,TYPE_ACCELEROMETER,0.050628662,4.552109,9.074799,2
2,1578462618693,TYPE_ACCELEROMETER,0.0015563965,4.462326,9.131668,2
3,1578462618713,TYPE_ACCELEROMETER,0.055419922,4.552704,8.652237,2
4,1578462618733,TYPE_ACCELEROMETER,-0.029571533,4.6341095,8.662399,2


In [21]:
name    = '5a0546857ecc773753327266'
floor   = 'B1'
file    = '5e15730aa280850006f3d005.txt'
columns = ['time', 'data tipe','Coordinate x (meter)', 'Coordinate y (meter)']
typ     = 'TYPE_WAYPOINT'
simple_data(typ, name, floor, file, columns)

,time,data tipe,Coordinate x (meter),Coordinate y (meter)
0,1578462618392,TYPE_WAYPOINT,230.03738,153.49635
1,1578462628512,TYPE_WAYPOINT,231.4029,158.41515
2,1578462638947,TYPE_WAYPOINT,232.462,164.41673
3,1578462649660,TYPE_WAYPOINT,233.94418,171.41417


In [22]:
name    = '5a0546857ecc773753327266'
floor   = 'B1'
file    = '5e15730aa280850006f3d005.txt'
columns = ['time', 'data tipe','ssid', 'bssid', 'RSSI', 'frequency', 'last seen timestamp']
typ     = 'TYPE_WIFI'
simple_data(typ, name, floor, file, columns)

,time,data tipe,ssid,bssid,RSSI,frequency,last seen timestamp
0,1578462618826,TYPE_WIFI,da39a3ee5e6b4b0d3255bfef95601890afd80709,c08ad78a45798cfe176a42b35c7381ae602711c5,-46,5825,1578462603277
1,1578462618826,TYPE_WIFI,7182afc4e5c212133d5d7d76eb3df6c24618302b,4d89139ca69acc0a8a762672a822411a769ac266,-49,5825,1578462618272
2,1578462618826,TYPE_WIFI,d839a45ebe64ab48b60a407d837fb01d3c0dfef9,30f85a5e14351468a6dd13718a9da3b0d7b73685,-49,5825,1578462618268
3,1578462618826,TYPE_WIFI,b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,fd0bdf5a4dca2566935b14a78c441846b4fbda57,-49,5825,1578462618270
4,1578462618826,TYPE_WIFI,b7e6027447eb1f81327d66cfd3adbe557aabf26c,bce435ee12b29ad4d543e1418e48fbdea5dfcce2,-49,5825,1578462618271


In [23]:
name    = '5a0546857ecc773753327266'
floor   = 'B1'
file    = '5e15730aa280850006f3d005.txt'
columns = ['time', 'data tipe',
           'UUID',	'MajorID',
           'MinorID',	'Tx Power',	
           'RSSI',	'Distance',
           'MAC Address','time']
typ     = 'TYPE_BEACON'
simple_data(typ, name, floor, file, columns)

,time,data tipe,UUID,MajorID,MinorID,Tx Power,RSSI,Distance,MAC Address,time
0,1578462618698,TYPE_BEACON,d9c573b719a17da4836208fc436f87b5ca1aa877,b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,-56,-58,1.2902861669921697,ff9a29cb2dcb4100880d02b305d5691e578e66cf,1578462618698
1,1578462618777,TYPE_BEACON,d9c573b719a17da4836208fc436f87b5ca1aa877,b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,-56,-75,8.66670689767102,a66e07d12078c184858797814ef872dbf8aa1af6,1578462618777
2,1578462618786,TYPE_BEACON,d9c573b719a17da4836208fc436f87b5ca1aa877,b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,-56,-76,9.586529272742789,6c8aeb5f05fa3b2458538c8af36d069fa2c54620,1578462618786
3,1578462618790,TYPE_BEACON,d9c573b719a17da4836208fc436f87b5ca1aa877,b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,-56,-63,2.3419357036317217,26145e606b575396f4ca1bc439d2a9b37fdc6fa0,1578462618790
4,1578462618791,TYPE_BEACON,d9c573b719a17da4836208fc436f87b5ca1aa877,b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,b6589fc6ab0dc82cf12099d1c2d40ab994e8410c,-56,-57,1.1423077651217026,ff9a29cb2dcb4100880d02b305d5691e578e66cf,1578462618791


## Data quality problem

Примечание о качестве данных: в обучающих файлах вы можете иногда обнаружить, что в строке отсутствует конечный символ новой строки, что приводит к переходу к следующей строке. Как решить эту проблему - решать вам. Этой проблемы нет в тестовых данных.

Кол-во строк: 26925
Нет последней строки: {'5cd56b83e2acfd2d33b5cab0B2': 0
Кол-во строк в начале меньше заявленного: 131
длина срок > 200 : 927

In [3]:
f1 = os.path.join(path, '5da138b74db8ce0c98bd4774/F3/5db299ab5741f4000680a7d3.txt')
with open(f1) as f:
    txt = f.readlines()

In [7]:
txt[:10]

['#\tstartTime:1571984628356\n',
 '#\tSiteID:5da138b74db8ce0c98bd4774\tSiteName:万象城\tFloorId:5da138ba4db8ce0c98bd4f61\tFloorName:F3\n',
 '1571984628363\tTYPE_WAYPOINT\t69.55722\t47.792942\n',
 '1571984628475\tTYPE_ACCELEROMETER\t-0.7484894\t-0.21789551\t11.977112\t2\n',
 '1571984628475\tTYPE_MAGNETIC_FIELD\t-6.1431885\t19.187927\t-35.90851\t3\n',
 '1571984628475\tTYPE_GYROSCOPE\t0.3887329\t0.01802063\t-0.0914917\t3\n',
 '1571984628475\tTYPE_ROTATION_VECTOR\t-5.021889E-4\t0.02278449\t-0.17875691\t3\n',
 '1571984628475\tTYPE_MAGNETIC_FIELD_UNCALIBRATED\t-58.27484\t-61.743164\t-321.74988\t-52.131653\t-80.93109\t-285.84137\t3\n',
 '1571984628475\tTYPE_GYROSCOPE_UNCALIBRATED\t0.25737\t0.0635376\t-0.14186096\t-3.3569336E-4\t-2.89917E-4\t2.4414062E-4\t3\n',
 '1571984628475\tTYPE_ACCELEROMETER_UNCALIBRATED\t-0.79696655\t-0.22686768\t11.796951\t0.0\t0.0\t0.0\t3\n']

В начале всего две строки

In [45]:
f1 = os.path.join(path, '5da138b74db8ce0c98bd4774/F3/5db299ab5741f4000680a7d3.txt')
dct = {}
with open(f1) as f:
    count = 0
    txt = f.readlines()
    for t in txt:
        if '#' in t:
            count += 1

In [12]:
f1.split('/')[-3]

'5da138b74db8ce0c98bd4774'

In [17]:
all_files = glob.glob('../input/train/*/*/*')

In [19]:
len(all_files)

26925

In [49]:
dct = {}
for file in all_files[:1000]:
    f1 = file.split('/')
    with open(file) as f:
        count = 0
        txt = f.readlines()
        for t in txt[:20]:
            if '#' in t:
                count += 1
        if count < 7:
            dct[f1[-3] + f1[-2]] = count

In [50]:
dct

{'5cd56c29e2acfd2d33b6d915B1': 7,
 '5cd56c29e2acfd2d33b6d915F6': 7,
 '5cd56c29e2acfd2d33b6d915L1': 7,
 '5cd56c29e2acfd2d33b6d915F2': 7,
 '5cd56c29e2acfd2d33b6d915F5': 7,
 '5cd56c29e2acfd2d33b6d915B2': 7,
 '5cd56c29e2acfd2d33b6d915F4': 7,
 '5cd56c29e2acfd2d33b6d915F3': 7,
 '5dbc1d84c1eb61796cf7c010B1': 10,
 '5dbc1d84c1eb61796cf7c010F6': 10,
 '5dbc1d84c1eb61796cf7c010F2': 10,
 '5dbc1d84c1eb61796cf7c010F5': 10,
 '5dbc1d84c1eb61796cf7c010F7': 10,
 '5dbc1d84c1eb61796cf7c010F4': 10,
 '5dbc1d84c1eb61796cf7c010F3': 10,
 '5dbc1d84c1eb61796cf7c010F8': 10}

In [ ]:
f1 = os.path.join(path, '5da138b74db8ce0c98bd4774/F3/5db299ab5741f4000680a7d3.txt')
dct = {}
with open(f1) as f:
    count = 0
    txt = f.readlines()
    for t in txt:
        print(len(t))

In [52]:
dct = {}
for file in all_files:
    f1 = file.split('/')
    with open(file) as f:     
        txt = f.readlines()
        for t in txt:
            if len(t) > 150:
                dct[f1[-3] + f1[-2]] = len(t)

In [ ]:
dct

In [62]:
a = [i for i in dct.values() if i > 200 ]

In [63]:
len(a)

927